In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [2]:
df = pd.read_csv('phising.csv')

In [3]:
df

,having_IP_Address,URL_Length,Shortining_Service,having_At_Symbol,double_slash_redirecting,Prefix_Suffix,having_Sub_Domain,SSLfinal_State,Domain_registeration_length,Favicon,...,popUpWidnow,Iframe,age_of_domain,DNSRecord,web_traffic,Page_Rank,Google_Index,Links_pointing_to_page,Statistical_report,Result
0,-1,1,1,1,-1,-1,-1,-1,-1,1,...,1,1,-1,-1,-1,-1,1,1,-1,-1
1,1,1,1,1,1,-1,0,1,-1,1,...,1,1,-1,-1,0,-1,1,1,1,-1
2,1,0,1,1,1,-1,-1,-1,-1,1,...,1,1,1,-1,1,-1,1,0,-1,-1
3,1,0,1,1,1,-1,-1,-1,1,1,...,1,1,-1,-1,1,-1,1,-1,1,-1
4,1,0,-1,1,1,-1,1,1,-1,1,...,-1,1,-1,-1,0,-1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11050,1,-1,1,-1,1,1,1,1,-1,-1,...,-1,-1,1,1,-1,-1,1,1,1,1
11051,-1,1,1,-1,-1,-1,1,-1,-1,-1,...,-1,1,1,1,1,1,1,-1,1,-1
11052,1,-1,1,1,1,-1,1,-1,-1,1,...,1,1,1,1,1,-1,1,0,1,-1
11053,-1,-1,1,1,1,-1,-1,-1,1,-1,...,-1,1,1,1,1,-1,1,1,1,-1


In [4]:
#data preprocessing normalization already done
#lets split the data and build the model
X =df.drop('Result',axis=1)
y = df['Result']
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=42)

In [5]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, classification_report
import joblib

# Step 1: Define the model
rf_classifier = RandomForestClassifier(random_state=42)

# Step 2: Define hyperparameter grid
params = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10]
}

# Step 3: Perform Grid Search with 5-fold CV
grid_search = GridSearchCV(
    estimator=rf_classifier,
    param_grid=params,
    cv=5,
    n_jobs=-1,
    verbose=2
)
grid_search.fit(X_train, y_train)

# Step 4: Extract best model and parameters
best_params = grid_search.best_params_
best_estimator = grid_search.best_estimator_

# Step 5: Make predictions
final_predictor = best_estimator.predict(X_test)

# Step 6: Evaluate performance
final_accuracy = accuracy_score(y_test, final_predictor)
final_report = classification_report(y_test, final_predictor)


# Step 8: Print results
print("✅ Best Parameters:", best_params)
print("✅ Final Accuracy:", final_accuracy)
print("✅ Classification Report:\n", final_report)
print("✅ Model saved as 'rf_best_model.pkl'")

Fitting 5 folds for each of 36 candidates, totalling 180 fits
✅ Best Parameters: {'max_depth': None, 'min_samples_split': 2, 'n_estimators': 100}
✅ Final Accuracy: 0.9669832654907282
✅ Classification Report:
               precision    recall  f1-score   support

          -1       0.97      0.95      0.96       956
           1       0.96      0.98      0.97      1255

    accuracy                           0.97      2211
   macro avg       0.97      0.97      0.97      2211
weighted avg       0.97      0.97      0.97      2211

✅ Model saved as 'rf_best_model.pkl'


In [8]:
import pickle
import os

# Create the directory if it doesn't exist
os.makedirs('models', exist_ok=True)

# Save the trained model correctly
with open('models/rf_best_model.pkl', 'wb') as file:
    pickle.dump(best_estimator, file)